## 1: load water levels and coords

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
sys.path.append("..")

In [3]:
import pandas as pd
# import geopandas as gpd
import numpy as np
import os

import features.utils

from features.meteo import *

from importlib import reload

import xarray 
from siphon.catalog import TDSCatalog

from time import time
from datetime import datetime, timedelta

In [4]:
root_dir = "../../"
data_dir = os.path.join(root_dir, "datasets/")
processed_data_dir = os.path.join(data_dir, "processed_data/")

water_levels_path = os.path.join(data_dir, "hydro_2018-2020/new_data_all.csv")
posts_path = os.path.join(processed_data_dir, "asunp.pkl")

meteo_dir = os.path.join(data_dir, "meteo_new/")

In [5]:
meteo_loader = DirMeteoLoader(meteo_dir)
meteo = meteo_loader.load()
meteo.head(3)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/gldsn/.local/share/virtualenvs/aij20-flood-lIcseS5N/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-ad9ba6624e81>", line 2, in <module>
    meteo = meteo_loader.load()
  File "../features/meteo/meteo_loaders.py", line 18, in load
    meteo_data = self._load_concat(filepathes)
  File "../features/meteo/meteo_loaders.py", line 27, in _load_concat
    dfs = load_utils.read_csvs(filepathes)
  File "../features/meteo/load_utils.py", line 33, in read_csvs
    df = reduce_memory_usage(df)
  File "../features/utils.py", line 49, in reduce_memory_usage
    df[col] = df[col].astype(np.float32)
  File "/home/gldsn/.local/share/virtualenvs/aij20-flood-lIcseS5N/lib/python3.8/site-packages/pandas/core/frame.py", line 3044, in __setitem__
    self._set_item(key, value)
  File "/home/gldsn/.local/share/virtualenvs/aij20-flood-lIcseS5N/lib

TypeError: object of type 'NoneType' has no len()

In [ ]:
def object_cols2num(df):
    """Tries to convert each objct col to num dtype, if error occures for
    some col, it's returned not modified"""
    object_cols_mask = (df.dtypes == "O").values
    object_cols = df.columns[object_cols_mask]
    
    for col in object_cols:
        df[col] = df[col].astype(np.float32, errors="ignore")
    return df

In [ ]:
water_levels = pd.read_csv(water_levels_path, sep=";")
posts = features.utils.load_pickle(posts_path)
posts = object_cols2num(posts)

water_levels = features.utils.reduce_memory_usage(water_levels)

water_levels.head(3)
posts.head(3)

## 2: form water_levels, manipulating with indexes 

In [ ]:
pd.to_datetime(water_levels["time"], yearfirst=True)

In [ ]:
water_datetime_date = pd.to_datetime(water_levels["time"], yearfirst=True).dt.date
water_levels["time"] = water_datetime_date

water_levels.rename(columns={"time": "date", "identifier": "id"}, inplace=True)

water_levels.sort_values(by=["id", "date"], inplace=True)
water_levels.set_index(["id", "date"], inplace=True)

water_levels.head()

## 3: get coords of hydrostations

In [ ]:
def coords_by_col_val(post_df, id_colname, needed_values):
    id_col_vals = post_df[id_colname]
    needed_rows_mask = id_col_vals.isin(needed_values)
    needed_rows = post_df[needed_rows_mask.values].reset_index(drop=True)
    
    coords_with_id_col = needed_rows[[id_colname, "lon", "lat"]]
    return coords_with_id_col

In [ ]:
needed_hydro_ids = water_levels.reset_index()["id"].unique()
hydro_coords = coords_by_col_val(posts, "gidro", needed_hydro_ids)
hydro_coords.head(2)

In [ ]:
hydro_coords.drop_duplicates(subset="gidro", inplace=True)
hydro_coords.rename(columns={"gidro": "id"}, inplace=True)
hydro_coords.set_index("id", inplace=True)

## 4: meteo processing

In [ ]:
meteo.head(2)

In [ ]:
dt_builder = DtBuilder()
meteo_dropper = DirMeteoDropper()
diff_cols = ["cloudCoverTotal", "windSpeed", "totalAccumulatedPrecipitation", "soilTemperature", "airTemperature", 
                        "relativeHumidity", "pressureReducedToMeanSeaLevel", "windAngleX", "windAngleY"]

In [ ]:
preprocessor = DirMeteoPreprocessor(meteo, meteo_dropper, dt_builder, diff_cols)
meteo_processed = preprocessor.preprocess()
meteo_processed.head()

In [ ]:
ordinal_extr_stats = [{"func": np.nanmean, "lag": 1, "winsize": 7}, 
                      {"func": np.nanmean, "lag": 7, "winsize": 30}, 
                      {"func": np.nanstd, "lag": 7, "winsize": 30}]

ordinal_cols = ["cloudCoverTotal", "windSpeed", "totalAccumulatedPrecipitation", "soilTemperature",
                            "airTemperature", "dewpointTemperature", "pressure", "pressureReducedToMeanSeaLevel",
                            "windAngleX", "windAngleY"]

min_extr_stats = [{"func": np.nanmean, "lag": 1, "winsize": 7}]
min_cols = ["minimumTemperatureAtHeightAndOverPeriodSpecified", "maximumTemperatureOverPeriodSpecified"]

#!!!!!!!!!!!!! add diff cols in min cols

config = ExtrConfig(meteo_processed.columns)
config.update_with_config_and_cols(ordinal_extr_stats, ordinal_cols)
config.update_with_config_and_cols(min_extr_stats, min_cols)
extr_config = config.get_config()

In [ ]:
type(pd.DataFrame(np.arange(20).reshape(4, 5))[2])

In [ ]:
meteo_extract = MeteoExtractManager(meteo_processed, extr_config)

In [ ]:
meteo_extract.agg_daily()

In [ ]:
meteo_extract.stats()

In [ ]:
meteo_extract.get_data()

# get meteo forecast data

In [ ]:
def get_weather_forecast(points: list, required_data):
    url = "http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.xml"
    point_outputs = []
    
    client = get_latest_dataset_client(url)
    query = build_base_query(client, required_data)
    
    for _, point in points.iterrows():
        query = update_query_point(query, point)
        print("get data, point: ", point)#, query)
        t1 = time()
        out = client.get_data(query)
        print("Got data, time:", time() - t1)
        point_outputs.append(out)
    
    return point_outputs

def get_latest_dataset_client(catalog_url):
    gfs_cat = TDSCatalog(catalog_url)
    ncss_client = gfs_cat.latest.subset()
    return ncss_client


def build_base_query(client, required_data):
    query = client.query()
    
    query = query.variables(*required_data)
    query = query.all_times()
    query = query.accept("netCDF4")

    return query


def update_query_point(query, point): #also deletes old point
    
    return query

# forecast = get_weather_forecast
forecast_url = "http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.xml"
catalog = TDSCatalog(forecast_url)

In [ ]:
class ForecastMeteoLoader:
    def __init__(self, start_date, end_date, meteo_coords, retrieved_vars, varnames_table: pd.DataFrame, parser):
        """
        :param varnames_table: df with 1 col containing source name and 2 col with forecast names
        :param meteo_coords: contains id, lon and lat cols
        """
        
        self.start_date = start_date
        self.end_date = end_date
        self.meteo_coords = meteo_coords
        self.parser = parser
        
        self.varnames_table = varnames_table
        
        self.forecast_url = "http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.xml"
        self.catalog = TDSCatalog(self.forecast_url)
        
        self.ncss_client = self._get_remote_client()
        self.retrieved_vars = self._convert_var_names(retrieved_vars)

    def load(self):
        raw_forecasts = self._download_forecast()
        
        points_df = []
        for point_forecast in raw_forecasts:
            df_point = self.parser.parse(point_forecast, self.retrieved_vars)
            points_df.append(df_point)
            
        
        df = self._build_general_df(points_df)
        df = self._filter_dates(df)
        print("set nortmal colnames")
        df = self._set_normal_colnames(df)
        return df
        
    def _get_remote_client(self):
        datasets = self.catalog.catalog_refs
        data_dates = self._extract_datasets_dates(datasets)
        idx_nearest_ds = self._nearest_date_idx(data_dates, start_date)
        nearest_dataset = datasets[idx_nearest_ds]
        
        #print(nearest_dataset, type(nearest_dataset))
        ncss_client = nearest_dataset.follow().datasets[0].subset() # clean later
        return ncss_client
    
    def _extract_datasets_dates(self, datasets):
        dates = []
    
        for ds_name in datasets.keys():
            str_date = ds_name.split("_")[3]
            ds_date = datetime.strptime(str_date, "%Y%m%d")

            dates.append(ds_date)

        return dates
    
    def _nearest_date_idx(self, dates, compared_date):
        time_diffs = []
    
        for data_date in dates:
            diff = abs(data_date - compared_date)
            time_diffs.append(diff)

        return np.argmin(time_diffs)
    
    def _download_forecast(self):
        point_outputs = []
        
        for _, point in meteo_coords.iterrows():
            query = self._build_base_query(point)
            print("get data", point)
            t1 = time()
            out = self.ncss_client.get_data(query)
            print("got data", time() - t1)
            point_outputs.append(out)
            
        return point_outputs
    
    def _build_base_query(self, point):
        query = self.ncss_client.query()
        #print(*self.retrieved_vars)
        #print(point["lon"], point["lat"])
        query.variables(*self.retrieved_vars)
#         query = query.all_times()
        query.time_range(self.start_date, self.end_date)
        query.lonlat_point(point["lon"], point["lat"])
        
        query.accept("netCDF4")

        return query
    
    def _convert_var_names(self, src_names):
        src2forecast_names = self.varnames_table.reset_index().set_index("src")
        new_names = [src2forecast_names.loc[name, "forecast"] for name in src_names]
        return new_names
    
    def _set_normal_colnames(self, df):
        forecast2src_names = self.varnames_table.reset_index().set_index("forecast")
        forecast_vars = forecast2src_names.index
        print('eeee')
        rename_dict = {forecast_name: forecast2src_names.loc[forecast_name, "src"] for forecast_name in forecast_vars}
        print("EEEEE", rename_dict)
        df.rename(columns=rename_dict, inplace=True)
        
        return df
    
    def _filter_dates(self, df):
        print("filter dates enter")
        datetime = pd.Index(df.reset_index()["datetime"]).tz_localize(None)
        print("mask created")
        in_needed_range_mask = (datetime >= self.start_date) & (datetime <= self.end_date)
        
        return df[in_needed_range_mask]
    
    def _build_general_df(self, dfs):
        print("build general")
        ids = list(self.meteo_coords["id"])
        print("aa")
        for station_id, df in zip(ids, dfs):
            df["id"] = station_id
        print("cc")
        general_df = pd.concat(dfs, axis=0)
        return general_df

In [ ]:
# TODO: check fill_forecast_nan() from meteo_extraction.ipynb
class ForecastParser:
    def parse(self, forecast, var_names):
        print("parsing")
        self.data = xarray.backends.NetCDF4DataStore(forecast)
    
        values = {}
        for name in var_names:
            var_value = self._var_val(name)
            values[name] = var_value
            
        index = self._get_timestamps()
        df = pd.DataFrame.from_dict(values, orient="columns")
        df.index = index
        
        return df
            
    def _var_val(self, name):
        values = self.data.get_variables()[name].values
        return self._reshape_var_val(values)
    
    def _reshape_var_val(self, vals):
        shape = vals.shape
        
        if len(shape) == 3:
            vals = vals[:, :, 0]
        elif len(shape) > 3:
            raise ValueError("В Forecast размерность values > 3")

        return vals.flatten()[:-1] # OMG WTF DOES THIS SLICE MEAN!? check later and clean code
    
    def _get_timestamps(self):
        attrs = self.data.get_attrs()
        start_time = pd.to_datetime(attrs["time_coverage_start"])

        hours_from_start = self.data.get_variables()["time"]
        time_deltas = self._timedelta_from_hours(hours_from_start)[:-1] #same slice 

        timestamps = start_time + time_deltas
        timestamps.name = "datetime"
        
        return timestamps
    
    def _timedelta_from_hours(self, hours):
        return pd.to_timedelta(hours.values.flatten(), "h") 

In [ ]:
class ForecastMeteoUnifier:
    def __init__(self):
        self.cloud_cover_col = "cloudCoverTotal"
        self.x_wind_col = 'windAngleX'
        self.y_wind_col = 'windAngleY'
        
        self.temperature_cols = ["airTemperature", "soilTemperature",
                                 "maximumTemperatureOverPeriodSpecified",
                                 "minimumTemperatureAtHeightAndOverPeriodSpecified",
                                 "dewpointTemperature"
                                ]
        
        self.pressure_cols = ["pressure", "pressureReducedToMeanSeaLevel"] 
    
    def unify(self, forecast_df):
        rescaled_df = self.rescale(forecast_df)
        return rescaled_df
        
    def rescale(self, df):
        df = self._rescale_cloud_cover(df)
        df = self._rescale_wind_vector(df)
        df = self._rescale_temperature(df)
        df = self._rescale_pressure(df)
        return df
        
    def _rescale_cloud_cover(self, df):
        df[self.cloud_cover_col] /= 10 # rescaling from 0-100 to 0-10
        return df

    def _rescale_wind_vector(self, df):
        wind_x = df[self.x_wind_col]
        wind_y = df[self.y_wind_col]
        vector_module = (wind_x ** 2 + wind_y ** 2)**0.5

        wind_x = np.sign(wind_x) * wind_x / vector_module
        wind_y = np.sign(wind_y) * wind_y / vector_module

        # if vector_module == 0, wind_x and wind_y are np.nan
        wind_x[vector_module == 0] = 0
        wind_y[vector_module == 0] = 0


        df[self.x_wind_col] = wind_x
        df[self.y_wind_col] = wind_y

        return df

    def _rescale_temperature(self, df):
        df[self.temperature_cols] -= 273 # temperature there is in absolute form
        return df

    def _rescale_pressure(self, df):
        df[self.pressure_cols] /= 100
        return df

In [ ]:
start_date = datetime.strptime("13-11-2020", "%d-%m-%Y")
end_date = start_date + timedelta(days=10)

parser = ForecastParser()

name_forecast_src = np.array([("Total_cloud_cover_entire_atmosphere_Mixed_intervals_Average", "cloudCoverTotal"),
                        ('u-component_of_wind_height_above_ground', "windAngleX"), 
                        ('v-component_of_wind_height_above_ground', "windAngleY"),
                        ('Wind_speed_gust_surface', "windSpeed"), 
                        ('Total_precipitation_surface_Mixed_intervals_Accumulation', "totalAccumulatedPrecipitation"), 
                        ("Temperature_height_above_ground", 'airTemperature'), 
                        ('Maximum_temperature_height_above_ground_Mixed_intervals_Maximum', 'maximumTemperatureOverPeriodSpecified'), 
                        ('Minimum_temperature_height_above_ground_Mixed_intervals_Minimum', 'minimumTemperatureAtHeightAndOverPeriodSpecified'),
                        ('Temperature_surface', 'soilTemperature'), 
                        ('Relative_humidity_height_above_ground', 'relativeHumidity'), 
                        ('Pressure_height_above_ground', 'pressure'), 
                        ('Pressure_reduced_to_MSL_msl', 'pressureReducedToMeanSeaLevel'),
                        ("Dewpoint_temperature_height_above_ground", "dewpointTemperature")
                       ])

retrieved_vars = list(name_forecast_src[:, 1])
varnames_table = pd.DataFrame(name_forecast_src, columns=["forecast", "src"])

In [ ]:
# TODO: сделать meteo_coords

meteo_coords = pd.DataFrame({"id": [30001, 30002], "lon": [123, 127], "lat": [53, 50]})

In [ ]:
forecast_loader = ForecastMeteoLoader(start_date, end_date, meteo_coords, retrieved_vars, varnames_table, parser)

In [ ]:
forecast = forecast_loader.load()

In [ ]:
forecast_unifier = ForecastMeteoUnifier()
forecast_unified = forecast_unifier.unify(forecast)

In [ ]:
forecast_unified.rename(columns={"id": "stationNumber"}, inplace=True)

In [ ]:
forecast_extractor = MeteoExtractManager(forecast_unified, extr_config)

forecast_extractor.agg_daily()
forecast_extractor.stats()
forecast_features = forecast_extractor.get_data()

In [ ]:
forecast_features.head()

In [ ]:
class Features:
    def __init__(self, meteo_loader, hydro_path):
        self.meteo_loader = meteo_loader
        self.hydro_path = hydro_path
    
    def load_from_file(self):
        self.meteo = self.meteo_loader.load()
        self.hydro = self._simple_load_hydro()
        
    def _simple_load_hydro(self):
        hydro = pd.read_csv(self.hydro_path, sep=";")
        hydro = features.reduce_memory_usage(hydro)
        
        hydro_date = pd.to_datetime(hydro["time"], yearfirst=True).dt.date
        hydro["time"] = hydro_date

        hydro.rename(columns={"time": "date", "identifier": "id"}, inplace=True)

        hydro.sort_values(by=["id", "date"], inplace=True)
        hydro.set_index(["id", "date"], inplace=True)
        
        return hydro
        
        
    # TODO: загрузка данных из файлов
    # TODO: предобработка и получение фич meteo и hydro 
    # TODO: мердж meteo и hydro
    # TODO: merge по коррелирующим и ближайшим
    # TODO: нахождение кластеров, аггрегация среднего global и по кластеру
    # TODO: создание фич для test в step predict
    # + прикрутить уже готовый data loader/ data retriever

In [ ]:
def E(a, b):
    for i in range(5):
        yield a+i, b+i*2

In [ ]:
E(1, 1)